In [1]:
!pip install stable-baselines3
!pip install shimmy
import numpy as np
import pandas as pd
import yfinance as yf
import os
import tensorflow as tf
from google.colab import drive
import gym
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv


^C


In [ ]:
# Descargar los precios
data = yf.download('AAPL', start='2011-9-29', end='2023-05-24', interval="1d")
# Imprimir data
#print(data.head())

# Obtener los precios de cierre y las fechas
prices = data['Adj Close'].values
dates = data.index.tolist()
prices2 = prices[30:]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[*********************100%%**********************]  1 of 1 completed


In [ ]:


# Función para calcular el RSI
def calculate_rsi(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n + 1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = [np.nan] * (n - 1) + [100. - 100. / (1. + rs)]

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # Cambio actual
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        rs = up / down
        rsi.append(100. - 100. / (1. + rs))

    return rsi



# Función para crear las secuencias de entrenamiento
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i:i + sequence_length]
        sequences.append(sequence)
    return np.array(sequences)





# Calcular el RSI
rsi = calculate_rsi(prices)

# Normalizar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
normalized_prices = scaler.fit_transform(prices.reshape(-1, 1))
normalized_rsi = scaler.fit_transform(np.array(rsi).reshape(-1, 1))

# Crear las secuencias de entrenamiento
sequence_length = 30
X_train = create_sequences(normalized_prices, sequence_length)
y_train = normalized_rsi[sequence_length:]

# Construir el modelo LSTM para predecir el RSI
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, 1)))
model.add(Dense(1, activation='sigmoid'))  # Capa con función de activación 'sigmoid'
model.compile(loss='mean_absolute_error', optimizer='adam')  # Cambiar a 'mean_absolute_error'

# Entrenar el modelo para predecir el RSI
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Realizar las predicciones del RSI
all_predictions = model.predict(X_train)

# Desnormalizar las predicciones del RSI para obtener los valores reales
all_predictions = scaler.inverse_transform(all_predictions)

# Imprimir los valores RSI predichos para todo el período
predicted_rsi = pd.Series(np.array(rsi[sequence_length:]), index=dates[sequence_length:])
predicted_rsi['Predicted_RSI'] = all_predictions.flatten()
#print(predicted_rsi)

# Imprimir los valores del RSI original y del RSI predicho con un bucle for
for date, rsi_value, predicted_rsi_value in zip(predicted_rsi.index, predicted_rsi, predicted_rsi['Predicted_RSI']):
    print(f"{date}: RSI={rsi_value:.2f}, Predicted_RSI={predicted_rsi_value:.2f}")

# Crear una lista con los valores predichos del RSI
predicted_rsi_values = np.array(predicted_rsi['Predicted_RSI'].tolist())

# Imprimir la lista de valores predichos del RSI
#print(predicted_rsi_values)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Epoch 1/100
91/91 [==============================] - 14s 53ms/step - loss: 0.1566
Epoch 2/100
91/91 [==============================] - 4s 43ms/step - loss: 0.1557
Epoch 3/100
91/91 [==============================] - 4s 39ms/step - loss: 0.1536
Epoch 4/100
91/91 [==============================] - 4s 45ms/step - loss: 0.1515
Epoch 5/100
91/91 [==============================] - 4s 40ms/step - loss: 0.1505
Epoch 6/100
91/91 [==============================] - 3s 32ms/step - loss: 0.1496
Epoch 7/100
91/91 [==============================] - 1s 16ms/step - loss: 0.1496
Epoch 8/100
91/91 [==============================] - 1s 15ms/step - loss: 0.1470
Epoch 9/100
91/91 [==============================] - 1s 16ms/step - loss: 0.1467
Epoch 10/100
91/91 [==============================] - 1s 15ms/step - loss: 0.1448
Epoch 11/100
91/91 [==============================] - 2s 20ms/step - loss: 0.1428
Epoch 12/100
91/91 [==============================] - 2s 25ms/step - loss: 0.1402
Epoch 13/100
91/91 [====

In [ ]:
#predicted_rsi_values

In [ ]:
# Calcular las métricas para las predicciones del modelo LSTM
capital_inicial = 100000
costo_inversion = 5000


# Inicializar variables y listas para el análisis de rendimiento
capital_actual = capital_inicial
posiciones = []
rendimientos = []

# Recorrer todas las fechas para realizar las operaciones de compra y venta
for i in range(40,len(dates) - sequence_length):
    date = dates[30:]
    rsi_value = predicted_rsi_values[i]

    if rsi_value < 40:
        signal = 'Compra'
    elif rsi_value > 65:
        signal = 'Venta'
    else:
        signal = 'Neutral'

    if signal == 'Compra':
        # Calcula la cantidad de acciones que puedes comprar con el costo de inversión fijo
        cantidad_acciones = int(costo_inversion / prices2[i])
        # Resta el costo de la inversión del capital disponible
        capital_actual -= costo_inversion
        # Agrega la posición a la lista de posiciones con el precio de compra
        posiciones.append({'Fecha': date, 'Cantidad': cantidad_acciones, 'Precio de compra': prices2[i]})

    elif signal == 'Venta':
        for posicion in posiciones:
            cantidad_acciones = posicion['Cantidad']
            precio_compra = posicion['Precio de compra']
            precio_venta = prices2[i]
            rentabilidad = (precio_venta - precio_compra) / precio_compra
            rendimientos.append(rentabilidad)

            capital_actual += cantidad_acciones * precio_venta

        posiciones = []

# Calcula el rendimiento acumulado
rendimiento_acumulado = (capital_actual - capital_inicial) / capital_inicial

# Calcula la rentabilidad total considerando todas las operaciones realizadas
rentabilidad_total = np.prod(1 + np.array(rendimientos))

# Imprime la rentabilidad total después de vender todas las acciones compradas
print("Rentabilidad total después de vender todas las acciones compradas:", rentabilidad_total)

# Imprime las métricas adicionales según lo que necesites
print("Métricas:")
print("Rendimiento acumulado:", rendimiento_acumulado)

# Número total de operaciones realizadas (compras y ventas)
num_operaciones = len(rendimientos)
print("Número total de operaciones:", num_operaciones)

# Número de operaciones ganadoras y perdedoras
num_operaciones_ganadoras = len([rendimiento for rendimiento in rendimientos if rendimiento >= 0])
num_operaciones_perdedoras = len([rendimiento for rendimiento in rendimientos if rendimiento < 0])
print("Número de operaciones ganadoras:", num_operaciones_ganadoras)
print("Número de operaciones perdedoras:", num_operaciones_perdedoras)

# Rendimiento promedio de operaciones ganadoras y perdedoras
rendimiento_promedio_ganadoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento >= 0])
rendimiento_promedio_perdedoras = np.mean([rendimiento for rendimiento in rendimientos if rendimiento < 0])
print("Rendimiento promedio de operaciones ganadoras:", rendimiento_promedio_ganadoras)
print("Rendimiento promedio de operaciones perdedoras:", rendimiento_promedio_perdedoras)

Rentabilidad total después de vender todas las acciones compradas: 93.10822222908874
Métricas:
Rendimiento acumulado: 0.2106845057296753
Número total de operaciones: 171
Número de operaciones ganadoras: 121
Número de operaciones perdedoras: 50
Rendimiento promedio de operaciones ganadoras: 0.08115536305628239
Rendimiento promedio de operaciones perdedoras: -0.08887767703106685


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
